In [1]:
import json, csv
from typing import List, Dict, Any
import numpy as np
from typing import Tuple, List
import time
from typing import List, Optional
import requests
from bs4 import BeautifulSoup
import re
from openai import OpenAI

from sentence_transformers import SentenceTransformer
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [6]:
# DO NOT NEED TO RUN THIS AGAIN

def write_json(path: str, lookup: Dict[str, Dict[str, Any]]) -> None:
    with open(path, "w", encoding="utf-8") as f:
        # write the entire lookup dict as a single JSON object
        json.dump(lookup, f, ensure_ascii=False, indent=2)

def write_csv(path: str, uuid_list: List[str]) -> None:
   with open(path, "w", encoding="utf-8", newline="") as f:
       w = csv.DictWriter(f, fieldnames=["row_id", "uuid"])
       w.writeheader()
       for i, u in enumerate(uuid_list):
           w.writerow({"row_id": i, "uuid": u})

def make_full_name(name, long_name):
   if not long_name:   # catches "" and None
       return name
   return name + " - " + long_name

def save_npy(path: str, matrix: np.ndarray) -> None:
   np.save(path, matrix)
   print(f"Saved matrix of shape {matrix.shape}, dtype {matrix.dtype} -> {path}")

In [2]:
def embed_normalize(texts: List[str], batch_size: int = 128) -> np.ndarray:
   emb = model.encode(
       texts,
       batch_size=batch_size,
       convert_to_numpy=True,
       normalize_embeddings=False,  # we’ll handle normalization ourselves
       show_progress_bar=True,
   ).astype(np.float32, copy=False)
   # L2-normalize rows
   norms = np.linalg.norm(emb, axis=1, keepdims=True)
   norms[norms == 0] = 1.0  # avoid division by zero
   emb = emb / norms
   return emb

In [7]:
# DONT NEED TO RUN THIS AGAIN - use uuid_list.csv, InventoryLookup.jsonl, and embedded_items.npy

# open inventory database
with open("inventory-5-31-25.json", "r", encoding="utf-8") as f:
   inventory_json = json.load(f)   # inventory_json is already a list of dicts

inventory_lookup = {}           # dict[uuid] -> {Name, Long name, Locations}
uuid_list = []                  # list of uuids, in EXACT row order
names_list = []                 # list of full_name strings, same order as UUID_list

for item in inventory_json:
    # inventory_json is a list of dicts, so item is a dict 

    # Build fast lookup (N-length dict)
    inventory_lookup[item["uuid"]] = {
        "name": item["name"],
        "long_name": item["long_name"],
        "locations": item["locations"]  # keep structured as list of dicts
    }

    # Build index-aligned lists
    uuid_list.append(item["uuid"])
    names_list.append(make_full_name(item["name"], item["long_name"]))


# Persist inventory_lookup 
write_json("InventoryLookup.json", inventory_lookup)  

# Persist uuid_list 
write_csv("uuid_list.csv", uuid_list)

# Vectorize names_list -> names_matrix (N x d), then L2-normalize rows
names_matrix = embed_normalize(names_list)           # shape: N x d, d = 384

# Persist names_matrix 
save_npy("embedded_items.npy", names_matrix)


Batches:   0%|          | 0/11 [00:00<?, ?it/s]

Saved matrix of shape (1343, 384), dtype float32 -> embedded_items.npy


In [3]:
with open("InventoryLookup.json", "r", encoding="utf-8") as f:
    inventory_lookup = json.load(f)

# Load uuid_list (list of uuids)
with open("uuid_list.csv", "r", encoding="utf-8") as f:
    reader = csv.reader(f)
    uuid_list = [row[1] for row in reader]  # each row is a single uuid

# Load names_matrix (numpy array of embeddings)
names_matrix = np.load("embedded_items.npy")

In [5]:

import re, time, requests
from typing import List, Tuple, Optional
from bs4 import BeautifulSoup

# --- Helpers ---

def _normalize_amazon_url(u: str) -> str:
    # strip trailing punctuation that often sneaks in from chat/markdown
    u = u.rstrip(").,;:]")
    # remove query params
    u = u.split("?", 1)[0]
    # collapse to canonical dp/ASIN form if present
    m = re.search(r"/dp/([A-Z0-9]{10})", u)
    if m:
        asin = m.group(1)
        return f"https://www.amazon.com/dp/{asin}"
    return u

_HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0 Safari/537.36"
    ),
    "Accept-Language": "en-US,en;q=0.9",
}

def _extract_title_from_html(html_text: str) -> Optional[str]:
    soup = BeautifulSoup(html_text, "html.parser")

    # 1) Amazon product title selectors
    for css in ["#productTitle", "#title", "span#productTitle"]:
        el = soup.select_one(css)
        if el:
            txt = el.get_text(strip=True)
            if txt:
                return txt

    # 2) Open Graph
    og = soup.find("meta", {"property": "og:title"})
    if og and og.get("content"):
        return og["content"].strip()

    # 3) Fallback <h1>
    h1 = soup.find("h1")
    if h1:
        txt = h1.get_text(strip=True)
        if txt:
            return txt

    # 4) Fallback <title>
    if soup.title:
        txt = soup.title.get_text(strip=True)
        if txt:
            return txt

    return None

# --- Core functions ---

def link_parser(urls: List[str], timeout: float = 10.0, sleep_sec: float = 0.4) -> List[str]:
    seen = set()
    titles: List[str] = []

    for raw_u in urls:
        u = _normalize_amazon_url(raw_u)
        if u in seen:
            continue
        seen.add(u)

        try:
            r = requests.get(u, headers=_HEADERS, timeout=timeout)
            # DEBUG: show status; uncomment next line to dump HTML if needed
            # print("DEBUG status:", r.status_code, "URL:", u)
            if r.status_code != 200 or not r.text:
                time.sleep(sleep_sec); continue

            title = _extract_title_from_html(r.text)
            if title:
                # Avoid adding useless 'Amazon.com' fallback as a signal
                if title.lower().strip() == "amazon.com":
                    # treat as failure; skip adding
                    pass
                else:
                    titles.append(title)

        except requests.RequestException as e:
            # DEBUG: print(e)  # uncomment if you want to see the error
            pass

        time.sleep(sleep_sec)

    return titles

def link_find_split(user_request: str) -> Tuple[str, List[str]]:
    url_pattern = re.compile(r"https?://\S+")
    urls = url_pattern.findall(user_request)
    user_request_text = url_pattern.sub("", user_request).strip()

    link_titles = link_parser(urls)
    return user_request_text, link_titles


In [6]:
def find_three_inventory_match(extracted_vector: np.ndarray,
                               names_matrix: np.ndarray,
                               uuid_list: List[str],
                               inventory_lookup: Dict[str, Dict]) -> List[Dict]:

    scores = names_matrix @ extracted_vector  # shape (N,)

    # Get top-3 indices
    top_idx = np.argpartition(scores, -3)[-3:]  # unsorted top-3
    top_idx = top_idx[np.argsort(scores[top_idx])[::-1]]  # sort descending

    three_possible_matches: List[Dict] = []
    for i in top_idx:
        uuid = uuid_list[i]
        item_info = inventory_lookup[uuid]
        full_name = item_info["name"]
        if item_info.get("long_name"):
            full_name += " — " + item_info["long_name"]

        three_possible_matches.append({
            "uuid": uuid,
            "full_name": full_name,
            "locations": item_info["locations"],
            "score": float(scores[i]),
        })

    return three_possible_matches



In [11]:
DEBUG_LLM = True  # flip on for a short run


_oai_client = OpenAI(api_key="sk-")

AREA_LABELS = [
    "Cabinet 1","Cabinet 2","Cabinet 3", "Cabinet 4", "Cabinet 5","Cabinet 6","Cabinet 7", "Cabinet 9", "Cabinet 11", "Cabinet 15", "Pegboard 1","Fabric", "LFP", "3D Printer Room",
    "Electronics", "Studio", "Spraypaint", "Cage/Crypt/Other"
]

def llm_match(user_request_text: str,
              links_items_list: List[str],
              three_possible_matches: List[Dict]) -> Tuple[str, str]:
    """
    Call an LLM to choose ONE candidate uuid and ONE area label.

    Returns:
        (item_uuid, item_location)
        If anything fails, returns ("", "Cage/Crypt/Other")
    """
    # Ensure scores are JSON-serializable
    for c in three_possible_matches or []:
        if "score" in c:
            c["score"] = float(c["score"])

    # --- System instructions ---
    system_instructions = (
        "You label makerspace restock requests.\n"
        "TASK:\n"
        "1) Pick exactly ONE inventory item from the provided CANDIDATES by its 'uuid'.\n"
        "2) Assign exactly ONE area label from:\n"
        f" {AREA_LABELS}.\n\n"
        "INPUT FIELDS YOU WILL RECEIVE:\n"
        "- user_request_text: free-text from the steward (links removed)\n"
        "- links_items_list: list of product titles parsed from any links\n"
        "- three_possible_matches: array of objects with {uuid, full_name, locations, score}\n\n"
        "RULES:\n"
        "- Only select a uuid that appears in three_possible_matches.\n"
        "- If one of the candidate items is only in the Cage or Crypt, it is most likely not the correct option.\n"
        "- If links_items_list contradict user_request_text, favor links_items_list unless clearly off-topic.\n"
        "- If candidates are near-ties, prefer the higher score.\n"
        "- Area mapping guidance:\n"
        " • If any location.container contains 'Cabinet', map to the number cabinet as listed in Area Labels\n"
        " • Map location.room (case-insensitive):\n"
        "   - 'fabric' → 'Fabric'\n"
        "   - 'lfp' → 'LFP'\n"
        "   - '3d printer room' / '3d printers' → '3D Printer Room'\n"
        "   - 'electronics' → 'Electronics'\n"
        "   - 'studio' → 'Studio'\n"
        "   - 'spraypaint' / 'spray booth' → 'Spraypaint'\n"
        "   - 'cage', 'crypt', 'main', 'shop', 'other' → 'Cage/Crypt/Other'\n\n"
        "OUTPUT:\n"
        "Return ONLY this JSON (no prose):\n"
        '{"item_uuid":"<candidate uuid>","item_location":"<one allowed area>"}'
    )

    user_payload = {
        "user_request_text": user_request_text or "",
        "links_items_list": list(links_items_list or []),
        "three_possible_matches": three_possible_matches or [],
    }

    schema = {
        "type": "object",
        "properties": {
            "item_uuid": {"type": "string"},
            "item_location": {"type": "string", "enum": AREA_LABELS},
        },
        "required": ["item_uuid", "item_location"],
        "additionalProperties": False,
    }

    try:
        # ---- CALL THE CHAT COMPLETIONS API ----
        # NOTE: 'response_format={"type": "json_object"}' instructs the model to return JSON,
        # but the JSON still arrives in choices[0].message.content (string), not as 'output_text'.
        resp = _oai_client.chat.completions.create(
            model="gpt-4o-mini",
            response_format={"type": "json_object"},
            messages=[
                {"role": "system", "content": system_instructions},
                {"role": "user", "content": json.dumps(user_payload, ensure_ascii=False)},
            ],
            temperature=0,
        )

        # --- Unified extraction of raw text (handles both APIs if you later switch back) ---
        raw: Optional[str] = getattr(resp, "output_text", None)  # Responses API
        if not raw:
            # Chat Completions API
            raw = (resp.choices[0].message.content or "").strip()

        if DEBUG_LLM:
            print("[LLM RAW][:300]:", (raw or "")[:300])

        if not raw:
            raise ValueError("Empty LLM response")

        # --- Parse JSON robustly ---
        # 1) Try direct JSON
        try:
            data = json.loads(raw)
        except json.JSONDecodeError:
            # 2) Fallback: extract first {...} block
            m = re.search(r"\{[\s\S]*\}", raw)
            if not m:
                raise
            data = json.loads(m.group(0))

        if DEBUG_LLM:
            print("[LLM PARSED]:", data)

        cand_uuids = {c.get("uuid") for c in (three_possible_matches or []) if c.get("uuid")}
        item_uuid = (data or {}).get("item_uuid")
        item_location = (data or {}).get("item_location")

        # --- Validate against candidates and allowed labels ---
        if item_uuid in cand_uuids and item_location in AREA_LABELS:
            if DEBUG_LLM:
                print("[LLM OK]:", item_uuid, "→", item_location)
            return item_uuid, item_location

        if DEBUG_LLM:
            print(
                "[VALIDATION FAIL]:",
                item_uuid,
                item_location,
                "; cand_uuids sample:",
                list(cand_uuids)[:3],
            )

    except Exception as e:
        print("[LLM ERROR]:", repr(e))  # <-- do not silently pass

    # Sensible fallback
    return "", "Cage/Crypt/Other"

In [14]:
# THE MAIN FOR LOOP 

with open("restocks-5-31-25.json", "r", encoding="utf-8") as f:
    restock_requests = json.load(f)

restocks_final_data = []

# i = 0
for req in restock_requests:
    # i += 1 
    # # if i == 5:
    #     break

    # Skip unapproved requests
    if req["is_approved"] == "false":
        continue

    print ("******************************************************")
    # Split text vs links
    print ("WHOLE MESSAGE =", req)
    print ("message = ", req["item"])
    user_request_text, links_items_list = link_find_split(req["item"])
    print ("\n text = ", user_request_text)
    print ("\n link items = ", links_items_list)
    

    # Build query string for embedding
    if len(links_items_list) != 0:
        # join all link titles into one string (can also append user text if you want)
        extracted_vector = embed_normalize(["".join(links_items_list)])[0]
    else:
        extracted_vector = embed_normalize([user_request_text])[0]

    # Find top-3 inventory matches
    three_possible_matches = find_three_inventory_match(extracted_vector, names_matrix, uuid_list, inventory_lookup)
    print ("\n matches =", three_possible_matches)

    # Ask LLM (or stub) to choose final item + area
    item_uuid, item_area = llm_match(user_request_text, links_items_list, three_possible_matches)

    print ("LLM RETURNED:")
    print (item_uuid)
    print (item_area)
    # Collect results (you can expand tuple into dict later for CSV/graph)
    restocks_final_data.append([item_uuid, item_area, req["timestamp_sent"], req["timestamp_completed"]])


******************************************************
message =  https://www.amazon.com/dp/B0BKSJYX9Q/

 text =  

 link items =  ['Gute 10Pcs Cast Acrylic Sheet,Colored Acrylic Sheet Plastic Sheet 8 x 8 Inch (.118" Thick) for Signs, DIY Display Projects, Craft and Easy to Cut（10 Colors,Transparent）']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


 matches = [{'uuid': 'be13f962-3b93-4dcb-a19f-3279cb527ec0', 'full_name': 'Plastic Sheeting — https://a.co/d/05u8q9V', 'locations': [{'room': 'Cage', 'container': '1a', 'specific': ''}], 'score': 0.5975993871688843}, {'uuid': '666279ae-15c8-4257-9a49-b97058051826', 'full_name': 'Printer — EPSON Color Printer', 'locations': [{'room': 'Cage', 'container': '8d', 'specific': None}], 'score': 0.5918001532554626}, {'uuid': '1de6fbb3-4f5f-488f-8587-bd6cfaba5a0c', 'full_name': 'Iron Rest Pad', 'locations': [{'room': 'Cage', 'container': '3b', 'specific': ''}], 'score': 0.5534443855285645}]
[LLM RAW][:300]: {"item_uuid":"be13f962-3b93-4dcb-a19f-3279cb527ec0","item_location":"Cage/Crypt/Other"}
[LLM PARSED]: {'item_uuid': 'be13f962-3b93-4dcb-a19f-3279cb527ec0', 'item_location': 'Cage/Crypt/Other'}
[LLM OK]: be13f962-3b93-4dcb-a19f-3279cb527ec0 → Cage/Crypt/Other
LLM RETURNED:
be13f962-3b93-4dcb-a19f-3279cb527ec0
Cage/Crypt/Other
******************************************************
message =  

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


 matches = [{'uuid': '801028a8-4677-44f9-9ffe-5ebb239fd2d2', 'full_name': 'Metal Charms', 'locations': [{'room': 'Main', 'container': 'Cabinet 3', 'specific': 'Shelf 6'}, {'room': 'Cage', 'container': '5c', 'specific': 'Stack 4|-3'}], 'score': 0.6526544094085693}, {'uuid': '85c93576-6216-4927-8e78-9f3ebd85e8e4', 'full_name': 'Pony Beads', 'locations': [{'room': 'Main', 'container': 'Cabinet 3', 'specific': 'Shelf 13'}, {'room': 'Cage', 'container': '5e', 'specific': '|4'}], 'score': 0.5851408839225769}, {'uuid': '677b6423-1af3-450b-882c-f5a73c980cf8', 'full_name': 'Iron-On Patches', 'locations': [{'room': 'Main', 'container': 'Cabinet 1', 'specific': 'Shelf 14'}], 'score': 0.5422393679618835}]
[LLM RAW][:300]: {"item_uuid":"801028a8-4677-44f9-9ffe-5ebb239fd2d2","item_location":"Cabinet 3"}
[LLM PARSED]: {'item_uuid': '801028a8-4677-44f9-9ffe-5ebb239fd2d2', 'item_location': 'Cabinet 3'}
[LLM OK]: 801028a8-4677-44f9-9ffe-5ebb239fd2d2 → Cabinet 3
LLM RETURNED:
801028a8-4677-44f9-9ffe-5eb

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


 matches = [{'uuid': 'afde04e1-4135-496c-903e-dd5a2d4a1334', 'full_name': 'Tacky Glue', 'locations': [{'room': 'Main', 'container': 'Cabinet 5', 'specific': 'Shelf 1'}, {'room': 'Cage', 'container': '5d', 'specific': 'Stack 4|1'}], 'score': 0.7459425330162048}, {'uuid': 'c57f9472-89ca-4b27-9df0-adbd9b34aa02', 'full_name': 'Leatherworking Tools', 'locations': [{'room': 'Main', 'container': 'Cabinet 9', 'specific': 'Shelf 4'}], 'score': 0.6460705399513245}, {'uuid': '0662010e-3d64-4111-956c-e1e2703ee704', 'full_name': 'Zippers', 'locations': [{'room': 'Main', 'container': 'Cabinet 1', 'specific': 'Shelf 14'}, {'room': 'Cage', 'container': '5c', 'specific': 'Stack 23|-3'}], 'score': 0.643201470375061}]
[LLM RAW][:300]: {"item_uuid":"afde04e1-4135-496c-903e-dd5a2d4a1334","item_location":"Cabinet 5"}
[LLM PARSED]: {'item_uuid': 'afde04e1-4135-496c-903e-dd5a2d4a1334', 'item_location': 'Cabinet 5'}
[LLM OK]: afde04e1-4135-496c-903e-dd5a2d4a1334 → Cabinet 5
LLM RETURNED:
afde04e1-4135-496c-90

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


 matches = [{'uuid': '7d2d9f6e-ec2d-4dbb-b1d4-db6824e4174d', 'full_name': 'Spring Clamps', 'locations': [{'room': 'Main', 'container': 'Cabinet 11', 'specific': 'Shelf 9'}, {'room': 'Backstock', 'container': 'Shelf 2', 'specific': '|9'}], 'score': 0.5485905408859253}, {'uuid': '7fe932a0-035c-44c4-a1f1-427692f87187', 'full_name': 'Cricut Lint Roller', 'locations': [{'room': 'Cage', 'container': '1d', 'specific': ''}, {'room': 'Backstock', 'container': 'Shelf 2', 'specific': '|1'}], 'score': 0.5218009948730469}, {'uuid': '0fdb1a6f-30bb-4c4b-899c-7f6ca2fb6e1a', 'full_name': 'Rubber Mallets — Fiber Glass Handle Rubber Mallets', 'locations': [{'room': 'Main', 'container': 'Cabinet 11', 'specific': 'Shelf 5'}, {'room': 'Backstock', 'container': 'Shelf 2', 'specific': '|2'}], 'score': 0.5062304139137268}]


KeyboardInterrupt: 

In [16]:
print (len(restocks_final_data))

160


In [17]:
print(restocks_final_data[:20])

[['be13f962-3b93-4dcb-a19f-3279cb527ec0', 'Cage/Crypt/Other', '1695866482.891472', 1695916051.055505], ['801028a8-4677-44f9-9ffe-5ebb239fd2d2', 'Cabinet 3', '1695956194.842946', 1696281457.775988], ['a3a6b195-f3f7-49e8-9b00-bb84948844a6', 'Cabinet 1', '1696041184.028703', 1696271601.443851], ['3e00df33-2c77-4be9-ad16-81f61734cee6', 'Cabinet 1', '1696106783.537074', 1696279071.421318], ['a3a6b195-f3f7-49e8-9b00-bb84948844a6', 'Cabinet 1', '1696106820.441048', 1696279043.672329], ['a3a6b195-f3f7-49e8-9b00-bb84948844a6', 'Cabinet 1', '1696146041.607456', 1696271580.881537], ['f425db72-a9c6-4fbd-a3a8-38afab8adbb4', 'Pegboard 1', '1696300261.608438', 1696614972.130854], ['6b40e62f-3af4-4099-8fe5-cf2b4464ec17', 'Pegboard 1', '1696300462.954466', 1696618824.391257], ['414b1655-517c-492f-a564-97f119358da8', 'Cabinet 1', '1696460698.281946', 1696614250.010368], ['25033e881a4e4dafbc4c9fd3be5875f3', 'Studio', '1696461108.810528', 1697763497.52571], ['368a7a823a9d402ba54e899e86afa00e', 'Pegboard 1

In [19]:
i = 0
for req in restock_requests:
    i += 1 
    if i == 5:
        break

    # Skip unapproved requests
    if req["is_approved"] is False:\
        continue

    print ("******************************************************")
    # Split text vs links
    print ("WHOLE MESSAGE =", req)

******************************************************
WHOLE MESSAGE = {'_id': {'$oid': '65163ce2dab76fafc080ef10'}, 'uuid': '56406890-7482-4f94-8864-608e03fcbf0f', 'item': 'Slightly bigger seed beads (current ones are much smaller than 4 mm)\nhttps://www.amazon.com/Redtwo-Glass-Seed-Beads-Friendship/dp/B0C5JBWBXB/ref=sr_1_36?crid=3S6EKWKRROIDJ&keywords=seed%2Bbeads&qid=1695956100&sprefix=seed%2Bbeads%2Caps%2C400&sr=8-36&th=1', 'quantity': '1 kit?', 'reason': 'Out of Stock', 'user_uuid': '4b87d886bc2448a98c91b110ff3716a8', 'authorized_request': False, 'timestamp_sent': '1695956194.842946', 'timestamp_completed': 1696281457.775988, 'is_approved': True, 'completion_note': 'These should be here within a week or less. '}
******************************************************
WHOLE MESSAGE = {'_id': {'$oid': '6518891fdab76fafc059ad8e'}, 'uuid': 'c8a2187c-a7a5-4881-bb95-78d93f961710', 'item': 'fabric stabilizer', 'quantity': '1', 'reason': 'Out of Stock', 'user_uuid': '748d01865e514a7893065c